In [1]:
import os
import inspect
import seaborn
import matplotlib
import matplotlib.pyplot as plt
import torch
import scanpy as sc
import tqdm
import sys
import pickle
import PyComplexHeatmap as pch
import scvi
import IPython
import scipy
import pandas as pd
import numpy as np
import anndata
import gseapy
import statannotations
from statannotations.Annotator import Annotator

sc.settings.figdir=os.path.expanduser('~/WbFigures/DevAnalysis')
sc._settings.settings._vector_friendly=True

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

if torch.cuda.is_available():
    print("GPU is available")
    print("Number of GPUs:", torch.cuda.device_count())
    print("GPU Name:", torch.cuda.get_device_name(0))
else:
    print("GPU is not available")

sys.path.append('/home/matthew.schmitz/utils/mts-utils/')
from genomics import sc_analysis


import sys
import antipode
import antipode.antipode_model
import antipode.model_functions
import antipode.model_distributions
import antipode.model_modules
import antipode.train_utils
import antipode.plotting
import antipode.post

from antipode.post import uniqlist

/home/matthew.schmitz/Matthew/utils/miniforge3/envs/rapids-24.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


GPU is not available


In [2]:
%pprint

Pretty printing has been turned OFF


In [3]:
batch_key='batch_name'
discov_key='species'
layer_key='spliced'
MDE_KEY = "X_antipode_UMAP"
leaf_key = 'level_2'
anndata.settings.allow_write_nullable_strings = True

model_tag = '1.9.1.8.5_Dev_final_noQ_INP_moredim_regionsample'
model_path='/home/matthew.schmitz/Matthew/models/'+model_tag
adata = sc.read_h5ad(os.path.join(model_path,'analysis_inprogress.h5ad'))
antipode_model=antipode.antipode_model.ANTIPODE.load(model_path,adata=adata,prefix='p3_',device=device)
antipode_model.save_params_to_uns()
adata=antipode_model.adata_manager.adata
pstore = adata.uns['param_store']

INFO     Generating sequential column names                                                                        


/home/matthew.schmitz/Matthew/utils/miniforge3/envs/rapids-24.12/lib/python3.12/site-packages/pyro/params/param_store.py:334: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  s

In [4]:
adata

AnnData object with n_obs × n_vars = 1793824 × 16738
    obs: 'G2M_score', 'S_score', 'batch_name', 'general_region', 'log10_n_counts', 'phase', 'region', 'species', 'female', 'general_region_species', '_scvi_discov_ind', '_scvi_batch_ind', 'psi_0', 'psi_1', 'psi_2', 'q_score', 'level_0', 'level_1', 'level_2', 'antipode_cluster', 'kmeans', 'anno_level_2', 'initial_class', 'ic_extended', 'abc_subclass_markercor', 'juv_subclass_markercor', 'linnarson_extended', 'lamanno_class', 'lamanno_subclass', 'lamanno_cluster', 'neighborhood', 'curated_class_label', 'new_ccl', 'new_neighborhood', 'ccl_extended', 'new_neighborhood_extended', 'n_genes', 'percent_ribo', 'msregion', 'singlet', 'individual', 'timepoint', 'n_counts', 'doublet', 'dataset_name', 'log_abs_sum_dynam', 'clipped_psi_2', 'log_n_genes'
    var: 'highly_variable', 'highly_variable_intersection', 'highly_variable_nbatches', 'highly_variable_rank', 'mean', 'means', 'n_cells', 'std', 'variances', 'variances_norm'
    uns: '_scvi_mana

In [5]:
# aggr_means=antipode.model_functions.group_aggr_anndata(adata,[discov_key,'level_2'],layer=layer_key,normalize=True)
# aggr_sums=antipode.model_functions.group_aggr_anndata(adata,['level_2'],layer=layer_key,normalize=False,agg_func=np.sum)
# log_real_means=antipode.model_functions.safe_log_transform(aggr_means[0],aggr_sums[0].sum(-1)[np.newaxis,...,np.newaxis])
log_real_means,real_mean_levels=antipode.model_functions.get_real_leaf_means(adata,discov_key,leaf_key,layer=layer_key)
real_means=pd.DataFrame(log_real_means.mean(0),columns=adata.var.index,index=real_mean_levels[leaf_key])
real_means=real_means.loc[real_mean_levels[leaf_key],:]

100%|██████████| 232/232 [01:47<00:00,  2.16it/s]


In [15]:
ccl_aggr_means=antipode.model_functions.group_aggr_anndata(adata,[discov_key,'ccl_extended'],layer=layer_key,normalize=True)
ccl_aggr_sums=antipode.model_functions.group_aggr_anndata(adata,['ccl_extended'],layer=layer_key,normalize=False,agg_func=np.sum)
ccl_log_real_means=antipode.model_functions.safe_log_transform(ccl_aggr_means[0],ccl_aggr_sums[0].sum(-1)[np.newaxis,...,np.newaxis])


100%|██████████| 67/67 [01:45<00:00,  1.58s/it]


In [6]:
pstore = adata.uns['param_store']
discov_cluster_params,cluster_params, cluster_labels,var_labels, zero_mask,(prop_taxon, prop_locs,prop_discov_di,prop_discov_dm)=antipode_model.calculate_cluster_params(flavor='numpy')
keep_clusters = np.array([x in adata.obs['level_2'].unique() for x in cluster_labels ])
cluster_labels = np.array(cluster_labels)[keep_clusters]
zero_mask = zero_mask[:,keep_clusters,:]
cluster_index = np.array([int(x) for x in cluster_labels])
zero_mask = np.nan_to_num(zero_mask,posinf=np.nan)
discov_cluster_params = discov_cluster_params[:,cluster_index,:] * zero_mask
prop_locs = prop_locs[cluster_index,:]
prop_discov_di = prop_discov_di[:,cluster_index,:] * zero_mask
prop_discov_dm = prop_discov_dm[:,cluster_index,:] * zero_mask
discov_labels=adata.obs[antipode_model.discov_key].cat.categories
latent_labels=[str(x) for x in range(pstore['discov_dc'].shape[1])]
n_clusters=antipode_model.level_sizes[-1]

/allen/programs/celltypes/workgroups/rnaseqanalysis/EvoGen/Team/Matthew/code/scANTIPODE/antipode/antipode_mixins.py:445: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  zero_mask = (adata.obs.groupby(self.discov_key)[leaf_level].value_counts().unstack().loc[:,cluster_labels]>=cluster_count_threshold).to_numpy()
/allen/programs/celltypes/workgroups/rnaseqanalysis/EvoGen/Team/Matthew/code/scANTIPODE/antipode/antipode_mixins.py:446: RuntimeWarning: divide by zero encountered in divide
  return discov_cluster_params,cluster_params, cluster_labels,var_labels, 1/zero_mask[...,np.newaxis],(prop_taxon, prop_locs,prop_discov_di,prop_discov_dm)


In [8]:
clipped_expr_df = pd.DataFrame(np.clip(discov_cluster_params,a_min=-20,a_max=1e10).mean(0),columns=adata.var.index,index=cluster_labels)

In [9]:
quantile = 0.98
coefdf = antipode.post.get_conserved_quantile_markers(discov_cluster_params,columns=adata.var.index,index=cluster_labels, q=quantile,antimarker=False)
selected_markers = antipode.post.select_marker_genes(
    coefdf, clipped_expr_df, marker=True,
    threshold=np.log(5/1e6))
marker_gene_order=uniqlist([selected_markers[x] for x in adata.uns['dendrogram_level_2']['categories_ordered']])

In [16]:
ccl_coefdf = antipode.post.get_conserved_quantile_markers(ccl_log_real_means,columns=adata.var.index,index=adata.obs['ccl_extended'].cat.categories, q=quantile,antimarker=False)


100%|██████████| 67/67 [03:42<00:00,  3.32s/it]


In [14]:
adata.obs.loc[:,['level_2','level_1','new_neighborhood_extended','ccl_extended','region','abc_subclass_markercor']]

,level_2,level_1,new_neighborhood_extended,ccl_extended,region,abc_subclass_markercor
TAGTATGTAATA_e14-WT8,157,26,P,NPC,Ctx,320 Astro-OLF NN
TTCGCATAGGTT_e14-WT8,271,11,T,Ctx_IT,Ctx,309 CB PLI Gly-Gaba
CCTCCAATTGGC_e14-WT8,216,5,T,Ctx_Allo,Ctx,309 CB PLI Gly-Gaba
CACGATATCCAT_e14-WT8,373,41,NN,Endothelial,Ctx,329 ABC NN
CAGCCGCGCCCT_e14-WT8,214,5,T,Ctx_Allo,Ctx,309 CB PLI Gly-Gaba
...,...,...,...,...,...,...
TTTGGTTTCCCTTGTG_SRR13565274_GSM5047783_GW21-04_cerebellum_Homo_sapiens_RNA-Seq_kOut,398,29,DMR,RE_ATOH1_Granule,Cb,314 CB Granule Glut
TTTGGTTTCTGCGGCA_SRR13565274_GSM5047783_GW21-04_cerebellum_Homo_sapiens_RNA-Seq_kOut,222,40,DMR,RE_Gly,Cb,310 CBX Golgi Gly-Gaba
TTTGTCACAGACGCAA_SRR13565274_GSM5047783_GW21-04_cerebellum_Homo_sapiens_RNA-Seq_kOut,78,22,DMR,RE_EOMES_UBC,Cb,118 ADP-MPO Trp73 Glut
TTTGTCACATAACCTG_SRR13565274_GSM5047783_GW21-04_cerebellum_Homo_sapiens_RNA-Seq_kOut,108,38,P,Progen_OPC,Cb,326 OPC NN


In [13]:
adata.obs.columns

Index(['G2M_score', 'S_score', 'batch_name', 'general_region',
       'log10_n_counts', 'phase', 'region', 'species', 'female',
       'general_region_species', '_scvi_discov_ind', '_scvi_batch_ind',
       'psi_0', 'psi_1', 'psi_2', 'q_score', 'level_0', 'level_1', 'level_2',
       'antipode_cluster', 'kmeans', 'anno_level_2', 'initial_class',
       'ic_extended', 'abc_subclass_markercor', 'juv_subclass_markercor',
       'linnarson_extended', 'lamanno_class', 'lamanno_subclass',
       'lamanno_cluster', 'neighborhood', 'curated_class_label', 'new_ccl',
       'new_neighborhood', 'ccl_extended', 'new_neighborhood_extended',
       'n_genes', 'percent_ribo', 'msregion', 'singlet', 'individual',
       'timepoint', 'n_counts', 'doublet', 'dataset_name', 'log_abs_sum_dynam',
       'clipped_psi_2', 'log_n_genes'],
      dtype='object')

In [10]:
selected_markers

{0: 'FOXB1', 1: 'MED29', 2: 'SP8', 6: 'RABIF', 8: 'FZD10-AS1', 9: 'DCTPP1', 11: 'YWHAZ', 13: 'CCT6A', 14: 'TSHZ1', 15: 'SCRT2', 16: 'UBXN1', 19: 'CDC25B', 24: 'SLC16A6', 25: 'TRAPPC2', 26: 'CHST7', 27: 'LUM', 28: 'PRDM8', 31: 'ALYREF', 34: 'CCNO', 35: 'SRM', 36: 'GID8', 39: 'AJUBA', 41: 'CORIN', 42: 'GABRG2', 44: 'CAD', 45: 'INHBA', 47: 'RAD21', 48: 'GNG8', 49: 'NOP58', 50: 'EFNA2', 58: 'PROX1', 59: 'CALCRL', 61: 'SNX7', 62: 'FBLL1', 65: 'MDH1', 66: 'CX3CR1', 67: 'EIF1B', 69: 'SLC6A11', 71: 'TRAPPC2', 73: 'NTF3', 76: 'PDLIM7', 78: 'ZFHX4', 79: 'TMEM132C', 80: 'NBDY', 81: 'OLA1', 82: 'RSPO1', 83: 'TRAPPC2', 85: 'MYBL1', 87: 'WNT3', 88: 'UBE2S', 89: 'EIF3H', 90: 'SOSTDC1', 91: 'GFAP', 94: 'NR2F2', 96: 'ROBO3', 97: 'SERBP1', 98: 'MCIDAS', 100: 'PTCH1', 101: 'PDGFRA', 102: 'MYLK', 103: 'ACSL6', 104: 'ISL1', 108: 'MYC', 109: 'TRAPPC2', 111: 'PDZD11', 112: 'DDX31', 114: 'GAS2', 116: 'PFDN5', 117: 'TRAPPC2', 118: 'PIMREG', 120: 'UBE2V2', 121: 'TP73', 122: 'LMNB1', 123: 'LHX6', 124: 'RACK1', 1

In [11]:
coefdf

,TRAPPC2,C17orf49,C2orf68,C4orf19,C4orf54,C11orf58,C9orf24,C11orf1,C3orf80,C8orf33,...,ZW10,ZWILCH,ZWINT,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ASIP,CCDC198
0,-0.372538,-0.253029,-0.666332,-2.484574,-0.566978,-0.116801,-2.204294,-0.524150,-1.046877,-0.494489,...,-0.634458,-1.281075,-1.039370,-0.555655,-0.386801,-0.492599,-1.034181,-0.821047,-1.714158,-1.767640
1,-0.308917,-0.290666,-0.665656,-1.949094,-0.628884,-0.171775,-2.423887,-0.562154,-0.582024,-0.479024,...,-0.503906,-0.970146,-1.051725,-0.502962,-0.460542,-0.462685,-1.134745,-0.742455,-1.313092,-1.262529
2,-0.228588,-0.300823,-0.336561,-2.857441,-0.556463,-0.127702,-3.078704,-0.758076,-1.098588,-0.289459,...,-0.409458,-1.402513,-1.566278,-0.393342,-0.653528,-0.507970,-1.620214,-0.714019,-2.040072,-1.814482
6,-0.402192,-0.233066,-0.583554,-2.046477,-0.656744,-0.298640,-2.114796,-0.663250,-1.106621,-0.393551,...,-0.593605,-0.979246,-1.471757,-0.575624,-0.599983,-0.465162,-1.035159,-0.687485,-1.304722,-1.496459
8,-0.504078,-0.616601,-0.602913,-1.019199,-0.755342,-0.327094,-2.188931,-0.373406,-1.158091,-0.173556,...,-0.600193,-0.995621,-1.769102,-0.505452,-0.932108,-0.761712,-1.082626,-0.811206,-0.973598,-0.647918
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392,-0.291091,-0.212329,-0.528351,-2.275191,-0.586600,-0.251136,-2.305926,-0.633504,-0.958233,-0.340975,...,-0.565279,-1.098604,-1.445760,-0.512159,-0.537609,-0.418328,-1.506285,-0.690056,-1.497523,-1.722514
394,-0.334852,-0.174300,-0.528525,-2.541744,-0.556705,-0.147669,-2.436520,-0.622193,-0.950077,-0.350124,...,-0.507679,-1.144274,-1.129043,-0.479358,-0.555033,-0.489833,-1.280189,-0.691462,-1.580355,-1.528670
395,-1.136095,-1.219385,-0.339539,-2.942554,-0.462002,-1.264198,-3.715767,-1.457268,-0.896159,-1.306719,...,-0.244886,-1.412037,-2.020369,-0.229168,-0.216617,-0.183995,-1.811603,-0.026170,-1.869090,-1.828967
397,-0.208884,-0.304163,-0.628585,-1.832158,-0.551590,-0.105379,-1.910649,-0.394144,-0.831769,-0.311847,...,-0.498319,-1.000786,-1.088759,-0.562266,-0.395487,-0.526158,-1.261608,-0.748422,-1.646842,-1.537614


In [17]:
ccl_coefdf

,TRAPPC2,C17orf49,C2orf68,C4orf19,C4orf54,C11orf58,C9orf24,C11orf1,C3orf80,C8orf33,...,ZW10,ZWILCH,ZWINT,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ASIP,CCDC198
Hypo_Glut_4,-0.820873,-1.177970,-1.094952,-5.603813,-2.294325,-0.163394,-6.827879,-0.696181,-2.858164,-0.841368,...,-1.312536,-3.356310,-3.688303,-1.358644,-2.220021,-0.952205,-2.065985,-2.203243,-4.012577,-4.853777
DE-ME_GABA,-0.366670,-0.749033,-1.157578,-4.547653,-2.225616,-0.276089,-4.737088,-0.979512,-1.535116,-1.084131,...,-1.117136,-3.855460,-4.832050,-1.233255,-1.650649,-0.899614,-1.937440,-1.084174,-4.102415,-3.940735
LGE_FOXP2_TSHZ1,-0.383610,-0.992511,-0.615233,-5.260323,-1.343347,-0.434168,-6.165697,-2.145880,-4.061696,-0.442142,...,-1.060038,-2.465349,-5.369750,-0.286909,-2.178013,-0.733180,-2.498071,-0.698002,-3.404821,-4.246020
Ctx_IT,-0.594849,-0.804239,-0.738485,-5.805456,-1.333528,-0.458989,-6.558375,-1.233735,-1.982386,-0.536484,...,-1.369197,-3.195014,-5.432046,-0.700736,-1.955970,-0.626115,-2.375831,-0.896224,-3.815291,-4.493143
Progen_Astro-Ependymal,-0.923899,-1.082071,-1.177198,-1.605571,-2.317445,-0.704928,-1.591669,-0.730893,-3.498423,-0.204062,...,-1.314128,-1.684216,-3.020839,-0.848463,-2.210559,-1.894555,-1.671324,-1.549646,-4.035697,-2.440551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Hypo_Glut_2,-0.926633,-1.326576,-1.275196,-5.031530,-2.232918,-0.474176,-6.734744,-1.846870,-3.440171,-0.440405,...,-1.081448,-3.447381,-7.262494,-0.932041,-1.961726,-0.864031,-1.866808,-1.333600,-3.951169,-4.792369
RE_Gaba_Purkinje,-0.453021,-0.448345,-2.886758,-6.339574,-1.889843,-0.589934,-6.360331,-1.301616,-2.816903,-1.019424,...,-1.773852,-4.274704,-3.943422,-0.956561,-2.803904,-1.369598,-2.704681,-1.784678,-4.260201,-4.218660
MGE_CRABP1_MAF_GABA,-0.714993,-1.054392,-0.615061,-5.687375,-1.889750,-0.420267,-7.077633,-1.908258,-4.264877,-0.666494,...,-1.159883,-3.354592,-5.513024,-0.875563,-2.215279,-0.823422,-3.165290,-1.018773,-3.608001,-4.449201
Hypo_Glut_1,-0.249886,-0.925681,-1.110769,-5.157996,-2.031756,-0.225042,-8.171231,-0.965551,-4.734511,-0.831364,...,-1.372312,-3.431541,-4.754654,-1.150761,-3.767961,-1.199426,-2.256195,-1.517096,-4.077635,-4.918835


In [33]:
# Function to get mode value (first mode if there are multiple)
def get_mode(series):
    return series.mode().iloc[0] if not series.mode().empty else np.nan

# Function to return top n marker genes from a row (sorted descending by coefficient)
def get_top_n_markers(row, n=5):
    return list(row.sort_values(ascending=False).head(n).index)

# Function to calculate top n regions with proportions formatted as "region (prop%)"
def get_top_regions(series, n=3):
    counts = series.value_counts(normalize=True)
    top_n = counts.head(n)
    return ", ".join([f"{idx} ({prop:.1%})" for idx, prop in top_n.items()])

# Group obs by level_2
grouped = adata.obs.groupby('level_2')
vcs = adata.obs['level_2'].value_counts()
results = []
for level2, group in grouped:
    mode_level_1 = get_mode(group['level_1'])
    mode_new_neighborhood = get_mode(group['new_neighborhood_extended'])
    mode_ccl = get_mode(group['ccl_extended'])
    mode_abc = get_mode(group['abc_subclass_markercor'])
    top_regions = get_top_regions(group['region'], n=3)
    
    # Top 5 markers for level_2 from gene_markers
    if level2 in coefdf.index:
        top5_level2 = get_top_n_markers(coefdf.loc[level2])
    else:
        top5_level2 = []
    
    # Top 5 markers for the level_2's assigned ccl_extended from ccl_markers
    if mode_ccl in ccl_coefdf.index:
        top5_ccl = get_top_n_markers(ccl_coefdf.loc[mode_ccl])
    else:
        top5_ccl = []
    
    results.append({
        'level_2': level2,
        'level_1': mode_level_1,
        'new_neighborhood_extended': mode_new_neighborhood,
        'ccl_extended': mode_ccl,
        'abc_subclass_markercor': mode_abc,
        'n_cells':vcs[level2],
        'top_regions': top_regions,
        'top5_markers_level2': ", ".join(top5_level2),
        'top5_markers_ccl': ", ".join(top5_ccl)
    })

result_df = pd.DataFrame(results)
result_df = result_df.sort_values('ccl_extended')  # order by ccl_extended alphabetically

print(result_df)
result_df.to_csv(os.path.join(sc.settings.figdir,'cluster_metadata.csv'),sep='\t')

/scratch/fast/50132/ipykernel_731618/1481285248.py:16: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = adata.obs.groupby('level_2')


     level_2  level_1 new_neighborhood_extended ccl_extended  \
66       114       19                        NN          ABC   
136      220       28                        NN   Angioblast   
144      231        6                        NN        Astro   
208      357        9                         P        Astro   
14        26        6                        NN        Astro   
..       ...      ...                       ...          ...   
173      283        1                       DMR    Thal_Glut   
42        79       19                        NN         VLMC   
141      227       19                        NN         VLMC   
64       111        6                        NN   unassigned   
97       161        6                        NN   unassigned   

            abc_subclass_markercor  n_cells  \
66                      329 ABC NN     6891   
136                    333 Endo NN     5040   
144                319 Astro-TE NN     6171   
208                316 Bergmann NN     4798

In [30]:
level2

398

In [31]:
adata.obs['level_2'].value_counts()

level_2
265    55256
36     40998
224    38513
356    33600
35     33163
       ...  
289      670
127      611
101      566
199      559
307      368
Name: count, Length: 232, dtype: int64